**Hyperparameters**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/tmp_yolo_vein_data/dataset.zip" -d "/content/dataset/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/Swap_Rotate_Database/per10/dehydrated/left_hand/dorsal/target/IMG_20240319_162818.jpg  
  inflating: /content/dataset/Swap_Rotate_Database/faisal/hydrated/left_hand/wrist/yolo_vein_roi/1714594688676.jpg  
  inflating: /content/dataset/Swap_Rotate_Database/joy_5012/dehydrated/left_hand/wrist/target/IMG_20240330_174816.jpg  
  inflating: /content/dataset/Swap_Rotate_Database/faisal/hydrated/left_hand/wrist/yolo_vein_roi/1714594688837.jpg  
  inflating: /content/dataset/Swap_Rotate_Database/per3/hydrated/left_hand/dorsal/yolo_roi/frame_97.jpg  
  inflating: /content/dataset/Swap_Rotate_Database/muktadir/hydrated/left_hand/dorsal/yolo_vein_roi/1714594427540.jpg  
  inflating: /content/dataset/Swap_Rotate_Database/asif_musta_17/hydrated/left_hand/dorsal/yolo_vein_roi/IMG_20240326_204251.jpg  
  inflating: /content/dataset/Swap_Rotate_Database/muktadir/hydrated/left_hand/dorsal/yolo_vein_roi/1714594425687.jpg  


In [ ]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from os import path
from PIL import Image
import matplotlib.pyplot as plt
import keras.backend as K
from sklearn.model_selection import train_test_split
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
epochs = 20
batch_size = 3
margin = 1  # Margin for constrastive loss.

In [ ]:
base_dataset_path = "/content/dataset/Swap_Rotate_Database"
dataset_version = "/"

landmarks = ["dorsal", "wrist"]

! ls /content/dataset/Swap_Rotate_Database

abdullah_band	farsi1		   mujahid	per14  per84	    sadat_band	     tm_band
alamin_17_band	farsi2		   muktadir	per15  per85	    sakib_emn2_band  towhid
apurbo		fazle_rabbi_5011   nafiz	per2   per86	    salman	     tuli
araf_5005	hasan_srbd	   nakib_band	per22  per87	    shahan	     tushar_5003
asif18_band	hasina		   niaz		per3   per88	    shakib_17_band   usha
asif_musta_17	huda_band	   nirjon_5004	per4   per89	    shakib_emn_band  zahid
bina		joy_5012	   nurul_band	per5   per9	    sharif_17_band   zakir_band
dear_band	junior		   pappu_band	per6   rabbi_5009   shirin
dipto_17	mahir_5003	   per1		per7   rakib_band   sizvy
emon_17_band	mansur		   per10	per8   rigan	    tahmid_srbd
emon_5001	mehrab_band	   per11	per81  rmina	    tamim_17_band
emon_5011	monzur_srbd	   per12	per82  robin_khalu  tanmoy_17
faisal		mosto_sharif_band  per13	per83  ruman_5011   tanvir_5001


In [ ]:
import os
def list_files_in_directory(directory):
  files = []
  for file in os.listdir(directory):
    # Join the directory path with the file name to get the full path
    full_path = os.path.join(directory, file)
    if os.path.isfile(full_path):
      files.append(full_path)
  return files

def get_file_path_list(dir):
  # Get a list of all files in the current directory
  all_files = list_files_in_directory(dir)

  # Get a list of all file paths in the current directory
  all_file_paths = [os.path.join(dir, file) for file in all_files]

  if len(all_file_paths) == 0:
    return []

  return all_file_paths

In [ ]:
# get all the folder name in a list from base directory
users = os.listdir(base_dataset_path + dataset_version)

exclude_users = ["sizvy", "hasina", "mansur"]

exclude_bad = ["junior", "mehrab_band", "per13", "per81"]

exclude_avg = ["nakib_band", "bina", "Rmina", "abdullah_band", "apurbo", "Faisal", "joy 5012", "Muktadir", "per15", "per3", "per83", "rabbi 5009", "Rakib_band", "salman_rootmate_2", "Tahmid_srbd", "tanmoy 17", "tuli"]

exclude_extra = ["emon_17_band", "per3", "per86", ]

users = [user for user in users if user not in exclude_users]

users = [user for user in users if user not in exclude_bad]

users = [user for user in users if user not in exclude_avg]

users = [user for user in users if user not in exclude_extra]

# shuffle
random.shuffle(users)

test_users = users[:8]
users = users[8:]

print(len(users))


60


In [ ]:
hydrated_list = []
dehydrated_list = []

# hands = ["left_hand", "right_hand"]
hands = ["left_hand"]
# landmarks = ["dorsal", "wrist"]
landmarks = ["dorsal"]

for user in users:
    for hand in hands:
        for landmark in landmarks:
            hydrated_list_url = base_dataset_path + dataset_version + user + "/" + "hydrated/" + hand + "/" + landmark + "/yolo_vein_roi"
            hydrated_list.append(get_file_path_list(hydrated_list_url))

            dehydrated_list_url = base_dataset_path + dataset_version + user + "/" + "dehydrated/" + hand + "/" + landmark + "/yolo_vein_roi"
            dehydrated_list.append(get_file_path_list(dehydrated_list_url))

In [ ]:
print(len(hydrated_list))
print(len(dehydrated_list))

60
60


In [ ]:
# List of image file paths
normal_images = hydrated_list
abnormal_images = dehydrated_list

tmp = []
for i in range(len(normal_images)):
  jj = 3
  if len(normal_images[i]) < 3:
    jj = len(normal_images[i])
    print(i, len(normal_images[i]))
#   if i==102:
#     jj = 1
  if i==4:
    jj = 2
  for j in range(jj):
    tmp.append(normal_images[i][j])

normal_images = tmp

print("-----------")

tmp = []
for i in range(len(abnormal_images)):
  jj = 3
  if len(abnormal_images[i]) < 3:
    jj = len(abnormal_images[i])
    print(i, len(abnormal_images[i]))
#   if i==35 or i==38:
#     jj = 1
#   if i==47:
#     jj = 1
  for j in range(jj):
    tmp.append(abnormal_images[i][j])

abnormal_images = tmp

print(len(normal_images))
print(len(abnormal_images))

# Split the data
train_normal, val_normal = train_test_split(normal_images, test_size=0.2, random_state=42)
train_abnormal, val_abnormal = train_test_split(abnormal_images, test_size=0.2, random_state=42)

# Combine lists and labels
train_images = train_normal + train_abnormal
train_labels = [0] * len(train_normal) + [1] * len(train_abnormal)
val_images = val_normal + val_abnormal
val_labels = [0] * len(val_normal) + [1] * len(val_abnormal)

# Convert to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
val_images = np.array(val_images)
val_labels = np.array(val_labels)


-----------
4 2
179
179


In [ ]:
# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=False
)

# Only rescale validation data
validation_datagen = ImageDataGenerator()

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    return image

def generator(images, labels, batch_size, is_training):
    datagen = train_datagen if is_training else validation_datagen
    num_samples = len(images)

    while True:
        for offset in range(0, num_samples, batch_size):
            batch_images = images[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]

            imgs = [load_and_preprocess_image(img) for img in batch_images]
            imgs = np.array([img.numpy() for img in imgs])

            augmented_imgs = datagen.flow(imgs, batch_size=batch_size, shuffle=False).next()

            yield augmented_imgs, np.array(batch_labels)

batch_size = 3
train_generator = generator(train_images, train_labels, batch_size, is_training=True)
validation_generator = generator(val_images, val_labels, batch_size, is_training=False)


In [ ]:
# Load the ResNet50 model
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the ResNet50 model
for layer in resnet50_base.layers:
    layer.trainable = False

# Freeze the layers of the VGG16 model
# for layer in vgg16_base.layers:
#     layer.trainable = False

# Add custom layers on top of ResNet50 base
resnet50_model = Sequential([
    resnet50_base,
    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    Dropout(0.1),
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

resnet50_model.compile(optimizer=Ad am(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
num_train_samples = len(train_images)
num_val_samples = len(val_images)

train_steps_per_epoch = math.ceil(num_train_samples / batch_size)
val_steps_per_epoch = math.ceil(num_val_samples / batch_size)

print(f"Number of training samples: {num_train_samples}")
print(f"Number of validation samples: {num_val_samples}")
print(f"Training steps per epoch: {train_steps_per_epoch}")
print(f"Validation steps per epoch: {val_steps_per_epoch}")

history_resnet50 = resnet50_model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=val_steps_per_epoch
)


Number of training samples: 286
Number of validation samples: 72
Training steps per epoch: 96
Validation steps per epoch: 24
Epoch 1/100
96/96 [==============================] - 8s 44ms/step - loss: 5.1014 - accuracy: 0.9406 - val_loss: 83.9106 - val_accuracy: 0.5000
Epoch 2/100
96/96 [==============================] - 3s 32ms/step - loss: 5.1493 - accuracy: 0.7832 - val_loss: 20.9990 - val_accuracy: 0.5000
Epoch 3/100
96/96 [==============================] - 3s 32ms/step - loss: 1.7899 - accuracy: 0.2273 - val_loss: 0.7044 - val_accuracy: 0.5000
Epoch 4/100
96/96 [==============================] - 3s 31ms/step - loss: 0.7047 - accuracy: 0.3916 - val_loss: 0.7025 - val_accuracy: 0.5000
Epoch 5/100
96/96 [==============================] - 3s 30ms/step - loss: 0.7030 - accuracy: 0.3531 - val_loss: 0.7020 - val_accuracy: 0.5000
Epoch 6/100
96/96 [==============================] - 3s 29ms/step - loss: 0.7010 - accuracy: 0.3531 - val_loss: 0.7016 - val_accuracy: 0.5000
Epoch 7/100
96/96 [==

In [ ]:
loss_resnet50, accuracy_resnet50 = resnet50_model.evaluate(validation_generator, steps=val_steps_per_epoch)
print(f'ResNet50 Validation Loss: {loss_resnet50}')
print(f'ResNet50 Validation Accuracy: {accuracy_resnet50}')


In [ ]:
resnet50_model.save('resnet50_vein_classification_model.h5')

In [ ]:
# # Load the VGG16 model
# vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# # Freeze the layers of the VGG16 model
# for layer in vgg16_base.layers:
#     layer.trainable = False

# # Add custom layers on top of VGG16 base
# vgg16_model = Sequential([
#     vgg16_base,
#     Flatten(),
#     Dense(512, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')
# ])

# vgg16_model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# history_vgg16 = vgg16_model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // train_generator.batch_size,
#     epochs=50,
#     validation_data=validation_generator,
#     validation_steps=validation_generator.samples // validation_generator.batch_size
# )


In [ ]:
# loss_vgg16, accuracy_vgg16 = vgg16_model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
# print(f'VGG16 Validation Loss: {loss_vgg16}')
# print(f'VGG16 Validation Accuracy: {accuracy_vgg16}')


In [ ]:
# vgg16_model.save('vgg16_vein_classification_model.h5')


In [ ]:
def plt_metric(history, metric, title, has_valid=True):
    plt.plot(history[metric])
    if has_valid:
        plt.plot(history["val_" + metric])
        plt.legend(["train", "validation"], loc="upper left")
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel("epoch")
    plt.show()

In [ ]:
# Plot the accuracy
plt_metric(history=history_resnet50.history, metric="accuracy", title="Model accuracy")

# Plot the constrastive loss
plt_metric(history=history_resnet50.history, metric="loss", title="Constrastive Loss")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the model
model = load_model('resnet50_vein_classification_model.h5')

In [ ]:
hydrated_list_test = []
dehydrated_list_test = []

hand = "left_hand"
landmark = "dorsal"

for user in test_users:
  hydrated_list_url = base_dataset_path + dataset_version + user + "/" + "hydrated/" + hand + "/" + landmark + "/ROI"
  hydrated_list_test.append(get_file_path_list(hydrated_list_url))

  dehydrated_list_url = base_dataset_path + dataset_version + user + "/" + "dehydrated/" + hand + "/" + landmark + "/ROI"
  dehydrated_list_test.append(get_file_path_list(dehydrated_list_url))

In [ ]:
# List of image file paths
normal_images_test = hydrated_list_test
abnormal_images_test = dehydrated_list_test

tmp = []
for i in range(len(normal_images_test)):
  for j in range(2):
    tmp.append(normal_images_test[i][j])

normal_images_test = tmp

tmp = []
for i in range(len(abnormal_images_test)):
  for j in range(2):
    tmp.append(abnormal_images_test[i][j])

abnormal_images_test = tmp

print(len(normal_images_test))
print(len(abnormal_images_test))

# Combine lists and labels
test_imgs = normal_images_test + abnormal_images_test
test_labels = [0] * len(normal_images_test) + [1] * len(abnormal_images_test)

# Convert to numpy arrays
test_imgs = np.array(test_imgs)
test_labels = np.array(test_labels)

In [ ]:
test_generator = generator(test_imgs, test_labels, batch_size, is_training=False)

test_step_epoch = math.ceil(len(test_imgs) / batch_size)
print(f"Test steps per epoch: {test_step_epoch}")

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_generator, steps=test_step_epoch)

print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')